In [1]:
import pandas as pd
from sqlalchemy import create_engine
import pymysql
pymysql.install_as_MySQLdb()
import sys
sys.path.append('../')

from config import remote_db_endpoint, remote_db_port
from config import remote_gwsis_dbname, remote_gwsis_dbuser, remote_gwsis_dbpwd
from config import local_gwsis_dbname, local_gwsis_dbuser, local_gwsis_dbpwd 

In [2]:
# Read in CSV file
csv_file = "Resources/crime.csv"
wiki_crime_df = pd.read_csv(csv_file)
wiki_crime_df.head()

,State,City,Population,Total Violent Crime,Murder,Rape,Robbery,Aggravated Assult,Total Property Crime,Burglary,Larceny-Theft,Motor Vehicle Theft,Arson
0,State,City,Population,Violent Crime,Property Crime,Arson2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Total,Murder and Nonnegligent Manslaughter,Rape1,Robbery,Aggravated Assault,Total,Burglary,Larceny-Theft,Motor Vehicle Theft,NaN,NaN,NaN,NaN
2,Alabama,Mobile3,248431,740.25,20.13,57.16,177.11,485.85,5453.83,1216.84,3730.21,506.78,22.94
3,Alaska,Anchorage,296188,1203.29,9.12,132.01,262.67,799.49,5415.82,748.17,3619.66,1047.98,20.93
4,Arizona,Chandler,249355,259.47,2.01,52.13,56.95,148.38,2329.61,314.41,1866.01,149.18,NaN


In [3]:
# Append Datafrome

new_crime_df = wiki_crime_df[['City', 'State', 'Population', 'Murder', 'Rape', 'Robbery', 'Aggravated Assult', 'Motor Vehicle Theft', 'Arson']].copy()
new_crime_df.head()

,City,State,Population,Murder,Rape,Robbery,Aggravated Assult,Motor Vehicle Theft,Arson
0,City,State,Population,Property Crime,Arson2,NaN,NaN,NaN,NaN
1,Murder and Nonnegligent Manslaughter,Total,Rape1,Aggravated Assault,Total,Burglary,Larceny-Theft,NaN,NaN
2,Mobile3,Alabama,248431,20.13,57.16,177.11,485.85,506.78,22.94
3,Anchorage,Alaska,296188,9.12,132.01,262.67,799.49,1047.98,20.93
4,Chandler,Arizona,249355,2.01,52.13,56.95,148.38,149.18,NaN


In [4]:
# Rename columns
crime_df = new_crime_df.rename(columns={"Aggravated Assult":"Assult", "Motor Vehicle Theft":"Car Theft"})
crime_df.head()

,City,State,Population,Murder,Rape,Robbery,Assult,Car Theft,Arson
0,City,State,Population,Property Crime,Arson2,NaN,NaN,NaN,NaN
1,Murder and Nonnegligent Manslaughter,Total,Rape1,Aggravated Assault,Total,Burglary,Larceny-Theft,NaN,NaN
2,Mobile3,Alabama,248431,20.13,57.16,177.11,485.85,506.78,22.94
3,Anchorage,Alaska,296188,9.12,132.01,262.67,799.49,1047.98,20.93
4,Chandler,Arizona,249355,2.01,52.13,56.95,148.38,149.18,NaN


In [5]:
# Cut dataframe rows
crime_df = crime_df.iloc[2:]

In [6]:
crime_df.head()

,City,State,Population,Murder,Rape,Robbery,Assult,Car Theft,Arson
2,Mobile3,Alabama,248431,20.13,57.16,177.11,485.85,506.78,22.94
3,Anchorage,Alaska,296188,9.12,132.01,262.67,799.49,1047.98,20.93
4,Chandler,Arizona,249355,2.01,52.13,56.95,148.38,149.18,NaN
5,Gilbert,Arizona,242090,2.07,16.11,21.07,46.26,55.76,12.39
6,Glendale,Arizona,249273,4.81,38.91,192.96,251.53,466.56,19.26


In [7]:
# Local Database Connection
rds_connection_string = "root:Daytona08!@localhost:3306/etl_project"
engine = create_engine(f'mysql://{rds_connection_string}')

In [8]:
#Check for table names
engine.table_names()

['craigslist_data', 'crime_data']

In [9]:
# Write to local database
crime_df.to_sql(name='crime_data', con=engine, if_exists='append', index=False)

In [10]:
# Call data from local database
pd.read_sql_query('select * from crime_data', con=engine).head()

,City,State,Population,Murder,Rape,Robbery,Assult,Car Theft,Arson
0,Mobile3,Alabama,248431,20.13,57.16,177.11,485.85,506.78,22.94
1,Anchorage,Alaska,296188,9.12,132.01,262.67,799.49,1047.98,20.93
2,Chandler,Arizona,249355,2.01,52.13,56.95,148.38,149.18,NaN
3,Gilbert,Arizona,242090,2.07,16.11,21.07,46.26,55.76,12.39
4,Glendale,Arizona,249273,4.81,38.91,192.96,251.53,466.56,19.26


In [11]:
# AWS Database Connection
awsengine = create_engine(f"mysql://{remote_gwsis_dbuser}:{remote_gwsis_dbpwd}@{remote_db_endpoint}:{remote_db_port}/etl_project")

# Create a remote database engine connection
awsconn = awsengine.connect()

# Write to AWS
crime_df.to_sql(name='crime_data', con=awsconn, if_exists='append', index=False)

In [14]:
# Call data from AWS Database
aws_crime_data = pd.read_sql("SELECT * FROM crime_data", awsconn)

aws_crime_data.head()

,City,State,Population,Murder,Rape,Robbery,Assult,Car Theft,Arson
0,Mobile3,Alabama,248431,20.13,57.16,177.11,485.85,506.78,22.94
1,Anchorage,Alaska,296188,9.12,132.01,262.67,799.49,1047.98,20.93
2,Chandler,Arizona,249355,2.01,52.13,56.95,148.38,149.18,NaN
3,Gilbert,Arizona,242090,2.07,16.11,21.07,46.26,55.76,12.39
4,Glendale,Arizona,249273,4.81,38.91,192.96,251.53,466.56,19.26
